In [ ]:
import os.path
import datetime
import pygetm.legacy
import pygetm
import cartopy.crs as ccrs

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

getm_cases = next(filter(os.path.isdir, ('/data/kb/getm-setups', '../../../getm-setups')))
igotm_data = next(filter(os.path.isdir, ('/server/data', '../../../igotm/data')))

northsea_path = os.path.join(getm_cases, 'NorthSea/Topo/NS6nm.v01.nc')  # path to legacy GETM North Sea test case
era_path = os.path.join(igotm_data, 'ERA-interim/20??.nc')              # path to ERA NetCDF file with t2m variable

map_proj = ccrs.PlateCarree()                                           # map projection to use for plotting

# Default time for plots
start = dt = datetime.datetime(2016,4,1,12)

In [ ]:
# Load GETM domain so we have a lon/lat grid to compute on/interpolate to.
domain = pygetm.legacy.domain_from_topo(northsea_path, nlev=1)
domain.initialize(pygetm.BAROTROPIC_2D)

# Load meteorology from ERA
airsea = pygetm.airsea.FluxesFromMeteo()
airsea.initialize(domain)
kwargs = {'preprocess': lambda ds: ds.isel(time=slice(4, -4)), 'chunks': {'time': 1}}
airsea.tcc.set(pygetm.input.from_nc(era_path, 'tcc', **kwargs))
airsea.t2m.set(pygetm.input.from_nc(era_path, 't2m', **kwargs) - 273.15)
airsea.d2m.set(pygetm.input.from_nc(era_path, 'd2m', **kwargs) - 273.15)
airsea.sp.set(pygetm.input.from_nc(era_path, 'sp', **kwargs))
airsea.u10.set(pygetm.input.from_nc(era_path, 'u10', **kwargs))
airsea.v10.set(pygetm.input.from_nc(era_path, 'v10', **kwargs))
domain.input_manager.update(start)

In [ ]:
# Compute and plot humidity
airsea.update_humidity(airsea.t2m)

fig, (ax1, ax2, ax3) = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj}, ncols=3, figsize=(16, 2.5))
(100 * airsea.ea / airsea.es).plot(ax=ax1)
airsea.qa.plot(ax=ax2)
airsea.rhoa.plot(ax=ax3)
ax1.coastlines()
ax2.coastlines()
ax3.coastlines();

In [ ]:
# Compute and plot longwave radiation
airsea.update_longwave_radiation(airsea.t2m)

fig, ax = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj})
airsea.ql.plot(ax=ax)
ax.coastlines();

In [ ]:
fig, (ax1, ax2) = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj}, ncols=2, figsize=(12, 3.5))
airsea.update_shortwave_radiation(dt)
airsea.albedo.plot(ax=ax1)
airsea.albedo_method = 2
airsea.update_shortwave_radiation(dt)
airsea.albedo.plot(ax=ax2)
ax1.coastlines()
ax2.coastlines();

In [ ]:
# Compute and plot heat and momentum fluxes
sss = domain.T.array(fill=35.)
airsea(dt, airsea.t2m, sss, calculate_heat_flux=True)

fig, ((ax1, ax2), (ax3, ax4)) = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj}, nrows=2, ncols=2, figsize=(12, 6))
airsea.taux.plot(ax=ax1)
airsea.tauy.plot(ax=ax2)
airsea.qe.plot(ax=ax3)
airsea.qh.plot(ax=ax4)
for ax in (ax1, ax2, ax3, ax4):
    ax.coastlines()

In [ ]:
# Set up plot of cloud cover and shortwave radiation at start time - we'll animate it later
fig, (ax1, ax2) = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj}, ncols=2, figsize=(12, 3))
pc_cloud = airsea.tcc.plot(ax=ax1, vmin=0, vmax=1)
pc_swr = airsea.swr.plot(ax=ax2, vmin=0, vmax=750)
ax1.coastlines()
ax2.coastlines()
title = fig.suptitle(start)

In [ ]:
# Animate cloud cover and shortwave radiation
for i in range(24 * 4 * 10):
    dt = start + datetime.timedelta(seconds=(i + 1) * 900)
    domain.input_manager.update(dt)
    airsea.shortwave_radiation(dt)
    pc_cloud.set_array(airsea.tcc.values.ravel())
    pc_swr.set_array(airsea.swr.values.ravel())
    title.set_text(dt)
    fig.canvas.draw()